# Washington Working Families Tax Credit

In 2008, the Washington State Legislature enacted the Working Families Tax Credit (WFTC, also called the Working Families Tax Exemption), but it was not funded until 2021, when Governor Jay Inslee signed [HB 1297](https://app.leg.wa.gov/billsummary?BillNumber=1297&Year=2021&Initiative=false) / [SB 5387](https://app.leg.wa.gov/billsummary?BillNumber=5387&Chamber=Senate&Year=2021).
The WFTC provides up to \$1,200 to filers who are eligible for the Earned Income Tax Credit, or who would be if their children had Social Security Numbers rather than Individual Taxpayer Identification Numbers.
Washington will deliver the first WFTC payments in 2023, based on 2022 tax returns.

## Examples

The Washington WFTC provided up to \$2,000 in 2021, for a joint filer with three children and earnings between about \$25,200 and \$26,100.


In [15]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px


def make_wftc(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="WA")
    sim.vary("employment_income", max=60_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            wa_wftc=sim.calc("wa_working_families_tax_credit")[0].round(),
            mtr=-sim.deriv("wa_working_families_tax_credit", "employment_income", wrt_target="head"),
            adults=adults,
            children=children,
        )
    )


# Make a table of EITCs for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_wftc(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    mtr="Marginal tax rate of Washington WFTC",
    adults="Adults",
    children="Children",
    wa_wftc="Washington WFTC",
)

fig = px.line(
    df,
    "employment_income",
    "wa_wftc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Washington Working Families Tax Credit",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

The NY EITC creates marginal tax rates ranging from -13.5 percent to +6.3 percent, plus downward spikes corresponding to the NY household credit cliffs.


In [13]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Washington Working Families Tax Credit marginal tax rate",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat=".1%", yaxis_range=[0, 0.2])
fig.show()

## Budgetary impact

Applying the Washington WFTC logic to the 2020 Current Population Survey March Supplement shows that the program would have cost an estimated \$79 million in 2020.

*NB: This is currently using the wrong phase-out start.*

[The Washington Department of Revenue estimated that the bill would provide \$236 million in refund payouts in FY2023](https://fnspublic.ofm.wa.gov/FNSPublicSearch/GetPDF?packageID=63327#page=6).


In [14]:
from openfisca_us import Microsimulation

ms = Microsimulation(year=2020)
(ms.calc("wa_working_families_tax_credit", period=2022).sum() / 1e6).round()

79.0